In [1]:
from openai import OpenAI
import os

os.environ.get("NAVIGATOR_TOOLKIT_API_KEY"),


client = OpenAI(
  api_key=os.environ.get("NAVIGATOR_TOOLKIT_API_KEY"),
  base_url="https://api.ai.it.ufl.edu/v1"
)

response = client.chat.completions.create(
  model="gpt-oss-120b", # model to send to the proxy
  messages = [
      { "role": "system", "content": "You are a helpful assistant." },
      {
          "role": "user",
          "content": "Write a haiku about an alligator."
      }
  ]
)
print(response.choices[0].message)

ChatCompletionMessage(content='Silent riverbank,  \nScales glint in the amber sun,  \nJaw snaps—still water.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning_content='We need to produce a haiku (5-7-5 syllable). The user wants a haiku about an alligator. Just produce it. Ensure correct syllable count. Let\'s craft:\n\nSilent riverbank,\nScales glint in the amber sun,\nJaw snaps—still water.\n\nCount: "Silent" (2), "riverbank" (3) =5. Good first line 5.\n\nSecond line: "Scales" (1) "glint" (1) "in" (1) "the" (1) "amber" (2) "sun" (1) =7? Let\'s count: Scales(1), glint(1) =>2, in(1)=>3, the(1)=>4, amber(2)=>6, sun(1)=>7. Good.\n\nThird line: "Jaw" (1) "snaps—still" (snaps=1, still=1) that\'s 2? But "snaps—still" together maybe counts as two syllables. "water" (2). So far: Jaw (1), snaps (1)=2, still (1)=3, water (2)=5. So 5 syllables. Good.\n\nMake sure punctuation doesn\'t affect. Provide haiku.')


In [2]:
import os
import time
import pandas as pd
from typing import Optional
from openai import OpenAI
from tqdm import tqdm


In [3]:
# ========= USER CONFIG =========

# When viewing on Windows
# Excel: use Data → Get Data → From Text/CSV → File Origin: UTF-8.
INPUT_TSV  = "train_df_distractors.tsv"     # <- your dataset file
OUTPUT_TSV = "train_df_with_cot.tsv"        # result
# MODEL      = "flux.1-schnell"        # proxy model name took me an 1hr 45 min to run
MODEL = "llama-3.1-70b-instruct" 
TEMP       = 0.2
MAX_TOKENS = 512
# =================================

In [4]:
# IMPORTANT: keep secrets in env vars
#   export NAVIGATOR_TOOLKIT_API_KEY="YOUR_KEY"
client = OpenAI(
    api_key=os.environ.get("NAVIGATOR_TOOLKIT_API_KEY"),

    base_url="https://api.ai.it.ufl.edu/v1"
)
from openai import OpenAI
client = OpenAI(base_url="https://api.ai.it.ufl.edu/v1", api_key=os.environ.get("NAVIGATOR_TOOLKIT_API_KEY"))

available = [m.id for m in client.models.list().data]
print("\n".join(sorted(available)))


codestral-22b
flux.1-dev
flux.1-schnell
gemma-3-27b-it
gpt-oss-120b
gpt-oss-20b
granite-3.3-8b-instruct
kokoro
llama-3.1-70b-instruct
llama-3.1-8b-instruct
llama-3.1-nemotron-nano-8B-v1
llama-3.3-70b-instruct
mistral-7b-instruct
mistral-small-3.1
nomic-embed-text-v1.5
sfr-embedding-mistral
whisper-large-v3


In [5]:

SYSTEM_PROMPT = (
    "You are a careful, faithful reasoning assistant. "
    "Given a question, context, and gold answer, produce a short, logical reasoning "
    "that JUSTIFIES the answer using ONLY the context. "
    "You MUST quote exact snippets from the context with double quotes. "
    "Keep the reasoning concise (2–6 sentences). "
    "Return output in one line, strictly as: "
    "##Reason: {reason} ##Answer: {answer}"
)

# Your requested instruction template
USER_TEMPLATE = """Question:
{question}

Context:
\"\"\"{context}\"\"\"

Gold Answer:
{answer}

Instruction: given the question, context, and answer above, provide a logical reasoning for that answer.
Please use the format of: ##Reason: {{reason}} ##Answer: {{answer}}.
Use quotes to directly refer to passages from the context in the reason.
"""

In [6]:
import json, time, random, traceback
from typing import Optional

def call_model(question: str, context: str, answer: str,
               max_retries: int = 5, base_delay: float = 1.5,
               debug: bool = False) -> Optional[str]:
    """
    Robust OpenAI API call with detailed input + API error logging.

    Returns model output text or None if all retries fail.
    Logs descriptive reasons for both input and API failures.
    """

    # ====== Input validation ======
    def check_input(name, value):
        if value is None:
            print(f"[input-error] {name} is None (missing value). Skipping this row.")
            return False
        if not isinstance(value, str):
            print(f"[input-error] {name} is not a string (type={type(value)}). Skipping this row.")
            return False
        if len(value.strip()) == 0:
            print(f"[input-error] {name} is empty after stripping whitespace.")
            return False
        return True

    ok_q = check_input("question", question)
    ok_c = check_input("context", context)
    ok_a = check_input("answer", answer)
    if not (ok_q and ok_c and ok_a):
        with open("api_failures.log", "a", encoding="utf-8") as logf:
            logf.write(f"\n[{time.strftime('%Y-%m-%d %H:%M:%S')}] "
                       f"INPUT ERROR | Q={ok_q} C={ok_c} A={ok_a} | "
                       f"question={repr(question)[:200]} context_len={len(str(context))} answer={repr(answer)}\n")
        return None

    if len(context) < 50:
        print(f"[warn] Context unusually short ({len(context)} chars). May produce weak reasoning.")
    if len(context) > 8000:
        print(f"[warn] Context very long ({len(context)} chars). May cause truncation or timeout.")

    # ====== Message assembly ======
    user_msg = USER_TEMPLATE.format(question=question, context=context, answer=answer)
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_msg},
    ]

    # ====== API call with retry ======
    for attempt in range(max_retries):
        try:
            resp = client.chat.completions.create(
                model=MODEL,
                temperature=TEMP,
                max_tokens=MAX_TOKENS,
                messages=messages,
            )

            if resp is None:
                print(f"[error] Null response (connection timeout?) on attempt {attempt+1}.")
                time.sleep(base_delay * (2 ** attempt))
                continue

            if debug:
                print(json.dumps(resp.to_dict(), indent=2))

            if not hasattr(resp, "choices") or not resp.choices:
                print(f"[error] No 'choices' in response on attempt {attempt+1}.")
                time.sleep(base_delay * (2 ** attempt))
                continue

            choice = resp.choices[0]
            message = getattr(choice, "message", None)
            text = getattr(message, "content", None)

            if text is None:
                finish_reason = getattr(choice, "finish_reason", "unknown")
                print(f"[warn] Got None content on attempt {attempt+1}. finish_reason={finish_reason}")
                time.sleep(base_delay * (2 ** attempt))
                continue

            text = " ".join(text.split())
            return text

        except Exception as e:
            err_type = type(e).__name__
            err_msg = str(e)

            if "Timeout" in err_msg or "Read timed out" in err_msg:
                cause = "network timeout"
            elif "429" in err_msg:
                cause = "rate limit (HTTP 429)"
            elif "ServiceUnavailable" in err_msg or "503" in err_msg:
                cause = "API overload"
            elif "ConnectionError" in err_msg or "Max retries exceeded" in err_msg:
                cause = "network connection problem"
            elif "invalid_request" in err_msg or "400" in err_msg:
                cause = "bad request (possibly too long context)"
            else:
                cause = "unknown API/SDK error"

            print(f"[warn] Attempt {attempt+1} failed: {err_type} ({cause})")
            print(f"   ↳ message: {err_msg[:200]}")

            with open("api_failures.log", "a", encoding="utf-8") as logf:
                logf.write(f"\n[{time.strftime('%Y-%m-%d %H:%M:%S')}] "
                           f"Attempt {attempt+1} | Cause: {cause}\n"
                           f"Inputs: Q={repr(question)[:100]} | "
                           f"C_len={len(context)} | A={repr(answer)}\n"
                           f"{traceback.format_exc()}\n")

            wait = base_delay * (2 ** attempt) + random.uniform(0, 0.5)
            print(f"   Retrying in {wait:.1f}s...\n")
            time.sleep(wait)

    print("[error] All retries failed. Giving up for this entry.")
    return None


In [7]:
df = pd.read_csv("train_df_distractors.tsv", sep="\t", dtype=str, keep_default_na=False)
df["cot"] = ""

for i in tqdm(range(len(df))):
    context, question, answer = df.at[i, "context"], df.at[i, "question"], df.at[i, "answer"]
    cot = call_model(question, context, answer)
    df.at[i, "cot"] = cot or ""

df.to_csv("train_df_with_cot.tsv", sep="\t", index=False)


  1%|          | 8/676 [00:24<30:37,  2.75s/it]

[warn] Context very long (10244 chars). May cause truncation or timeout.


  1%|▏         | 9/676 [00:35<56:25,  5.08s/it]

[warn] Context very long (10244 chars). May cause truncation or timeout.


  1%|▏         | 10/676 [00:41<1:02:30,  5.63s/it]

...

 99%|█████████▊| 666/676 [1:11:42<00:46,  4.65s/it]

[warn] Context very long (10337 chars). May cause truncation or timeout.


 99%|█████████▊| 667/676 [1:11:46<00:39,  4.40s/it]

[warn] Context very long (9590 chars). May cause truncation or timeout.


100%|██████████| 676/676 [1:12:43<00:00,  6.45s/it]
